In [2]:
import tensorflow as tf
import numpy as np

In [4]:
a = tf.placeholder(tf.float32,shape=(2,2))

In [8]:
b = tf.placeholder(tf.float32,shape=(2,2))

In [9]:
mul = tf.matmul(a,b)

In [12]:
with tf.Session() as session:
    output = session.run(mul,feed_dict={a:np.random.rand(2,2),b:np.random.rand(2,2)})
    print(output)

[[0.06972897 0.58849835]
 [0.39159548 0.8666266 ]]


In [13]:
X = tf.random_normal((2,2))

In [15]:
init = tf.global_variables_initializer()

In [16]:
with tf.Session() as session:
    session.run(init)
    print(X.eval())

[[-0.10892873 -1.317373  ]
 [ 0.26831132  1.3587077 ]]


In [22]:
u = tf.Variable(20.0)

In [23]:
cost = u**2 + u - 1

In [24]:
opt = tf.train.GradientDescentOptimizer(0.3).minimize(cost)

In [25]:
init = tf.global_variables_initializer()

In [27]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(5):
        sess.run(opt)
        print(u.eval())

7.7
2.7799997
0.8119998
0.024799883
-0.29008007


### nn

In [38]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

In [29]:
def get_transformed_digit():
    #import
    dt = pd.read_csv('digit.csv').values.astype(np.float32)
    np.random.shuffle(dt)
    X = dt[:,1:]
    Y = dt[:,0]
    
    #pca
    pca = PCA(n_components=100)
    Z = pca.fit_transform(X)
    exp_sum = np.sum(pca.explained_variance_ratio_)
    
    #normalize
    z_mean = np.mean(Z,axis=0)
    z_std = np.std(Z, axis = 0)
    Z_normalized = (Z-z_mean)/z_std
    
    #split
    X_train = Z_normalized[:-300]
    Y_train = Y[:-300]
    X_test = Z_normalized[-300:]
    Y_test = Y[-300:]
    
    #return
    return X_train, Y_train, X_test, Y_test

In [30]:
def y2indicator(y_vector):
    """
    transform the vector y to matrix y
    """
    l = len(y_vector)
    indi = np.zeros((l,10))
    for i in range(l):
        indi[i,int(y_vector[i])] = 1
    return indi

In [31]:
def error_rate(p,y):
    return np.mean(p!=y)

In [46]:
def tf_nn():
    X_train, Y_train, X_test, Y_test = get_transformed_digit()
    lr = 4*(1e-5)
    reg = 1e-2
    Y_train_matrix = y2indicator(Y_train)
    Y_test_matrix = y2indicator(Y_test)
    
    N,D = X_train.shape
    batch_size = 500
    batch_num = N // batch_size
    
    M = 100
    K = 10
    W1_init = np.random.rand(D,M)/np.sqrt(D)
    b1_init = np.random.rand(M)
    W2_init = np.random.rand(M,K)/np.sqrt(M)
    b2_init = np.random.rand(K)
    
    X = tf.placeholder(dtype=tf.float32, shape=(None, D))
    T = tf.placeholder(dtype=tf.float32, shape=(None, K))
    W1 = tf.Variable(W1_init.astype(np.float32))
    b1 = tf.Variable(b1_init.astype(np.float32))
    W2 = tf.Variable(W2_init.astype(np.float32))
    b2 = tf.Variable(b2_init.astype(np.float32))
    
    z = tf.nn.relu(tf.matmul(X,W1) + b1)
    a = tf.matmul(z,W2) + b2
    
    cost = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=T, logits=a))
    opt = tf.train.RMSPropOptimizer(learning_rate=0.00004,decay=0.9,momentum=0.999).minimize(cost)
    
    predict = tf.argmax(a,1)
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as s:
        s.run(init)
        for i in range(20):
            s.run(opt, feed_dict={X: X_train,T:Y_train_matrix})
            pred = s.run(predict,feed_dict={X: X_test})
            print(error_rate(Y_test,pred))
    
    
    

In [47]:
tf_nn()

0.8666666666666667
0.8633333333333333
0.8633333333333333
0.86
0.86
0.85
0.8466666666666667
0.83
0.8166666666666667
0.7766666666666666
0.76
0.7666666666666667
0.78
0.7766666666666666
0.7766666666666666
0.77
0.7433333333333333
0.7333333333333333
0.7233333333333334
0.7233333333333334
